In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)
!pip install transformers
!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install texttable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://pytorch-geometric.com/whl/torch-1.11.0+cu113.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#add /content in the end of next 6 lines to paste them in /content directory.(used for importing the GPT-GNN framework)
!cp /content/drive/MyDrive/GPT_GNN/__init__.py
!cp /content/drive/MyDrive/GPT_GNN/conv.py
!cp /content/drive/MyDrive/GPT_GNN/model.py
!cp /content/drive/MyDrive/GPT_GNN/utils.py
!cp /content/drive/MyDrive/GPT_GNN/data.py
!cp -av /content/drive/MyDrive/GPT_GNN

'/content/drive/MyDrive/GPT_GNN' -> '/content/GPT_GNN'
'/content/drive/MyDrive/GPT_GNN/__init__.py' -> '/content/GPT_GNN/__init__.py'
'/content/drive/MyDrive/GPT_GNN/conv.py' -> '/content/GPT_GNN/conv.py'
'/content/drive/MyDrive/GPT_GNN/model.py' -> '/content/GPT_GNN/model.py'
'/content/drive/MyDrive/GPT_GNN/utils.py' -> '/content/GPT_GNN/utils.py'
'/content/drive/MyDrive/GPT_GNN/data.py' -> '/content/GPT_GNN/data.py'
'/content/drive/MyDrive/GPT_GNN/__pycache__' -> '/content/GPT_GNN/__pycache__'
'/content/drive/MyDrive/GPT_GNN/__pycache__/conv.cpython-37.pyc' -> '/content/GPT_GNN/__pycache__/conv.cpython-37.pyc'
'/content/drive/MyDrive/GPT_GNN/__pycache__/utils.cpython-37.pyc' -> '/content/GPT_GNN/__pycache__/utils.cpython-37.pyc'
'/content/drive/MyDrive/GPT_GNN/__pycache__/__init__.cpython-37.pyc' -> '/content/GPT_GNN/__pycache__/__init__.cpython-37.pyc'
'/content/drive/MyDrive/GPT_GNN/__pycache__/data.cpython-37.pyc' -> '/content/GPT_GNN/__pycache__/data.cpython-37.pyc'
'/content/dri

In [ ]:
!pip install transformers
!pip install sacremoses

In [ ]:
from transformers import *

from collections import defaultdict
import torch
import data
import gensim
from gensim.models import Word2Vec
from tqdm import tqdm
import scipy.sparse as sp
import numpy as np
import dill
import pandas as pd
import utils

In [ ]:
cuda = 0
input_dir_user_features = "/content/drive/MyDrive/users_features.csv"
input_dir_merchant_features = "/content/drive/MyDrive/merchant_features.csv"
input_dir_edgelist = "/content/drive/MyDrive/edgelist.csv"
output_dir = "/content/drive/MyDrive/graph_FD.pk"
graph = data.Graph()

edge_df = pd.read_csv(input_dir_edgelist,index_col=0)
usersDF = pd.read_csv(input_dir_user_features,index_col=0)
merchantDF = pd.read_csv(input_dir_merchant_features,index_col=0)


In [ ]:
edge_df

,cc_num,name,type,time
0,630441765090,fraud_Pacocha-Bauch,small,2019
1,213126662687660,fraud_Doyle Ltd,huge,2019
2,60495593109,fraud_Volkman PLC,huge,2019
3,3518234918950662,fraud_Jaskolski-Dibbert,large,2019
4,213175487106448,fraud_Fisher Inc,small,2019
...,...,...,...,...
94871,3558652751678952,"fraud_Streich, Dietrich and Barton",small,2020
94872,30030380240193,"fraud_Wiza, Schaden and Stark",medium,2020
94873,3585740823295298,fraud_Cartwright-Harris,large,2020
94874,3502377050801561,"fraud_Zemlak, Tillman and Cremin",small,2020


In [ ]:
for index, row in tqdm(usersDF.iterrows(),total=usersDF.shape[0]):
  graph.node_forward['user'][row['id']] = index

for index, row in tqdm(merchantDF.iterrows(),total=merchantDF.shape[0]):
  graph.node_forward['merchant'][row['id']] = index

graph.node_feature['user'] = usersDF
graph.node_feature['merchant'] = merchantDF

100%|██████████| 693/693 [00:00<00:00, 14098.88it/s]


In [ ]:
merchantDF

,id,type,category,mean_amt,min_amt,1Q_amt,2Q_amt,3Q_amt,max_amt
0,fraud_Quitzon-Goyette,merchant,home,60.714255,1.25,22.830,51.100,83.015,257.87
1,fraud_Schmitt Ltd,merchant,misc_net,69.449684,1.01,4.530,13.510,92.935,557.21
2,"fraud_Schumm, McLaughlin and Carter",merchant,food_dining,49.612185,1.14,12.935,38.160,62.445,303.29
3,fraud_Parisian and Sons,merchant,gas_transport,62.017722,13.08,49.965,59.265,72.005,101.09
4,fraud_Bahringer-Larson,merchant,grocery_net,51.435500,11.38,37.675,50.625,60.850,108.85
...,...,...,...,...,...,...,...,...,...
688,fraud_Kirlin and Sons,merchant,personal_care,51.371520,1.06,14.080,37.050,63.370,318.16
689,"fraud_Nicolas, Hills and McGlynn",merchant,entertainment,67.716309,1.49,18.850,59.340,84.620,535.23
690,"fraud_Kunze, Larkin and Mayert",merchant,travel,106.371304,1.13,3.665,6.625,8.615,579.90
691,"fraud_Hermiston, Pacocha and Smith",merchant,health_fitness,51.842301,1.47,17.490,43.750,73.420,210.44


In [ ]:
for index,row in tqdm(edge_df.iterrows(),total=edge_df.shape[0]):
  user_id = row['cc_num']
  merchant_id = row['name']
  user_node = usersDF.loc[usersDF['id']==user_id].to_dict(orient='index') #hoping to be a pd.Series
  user_node = user_node[list(user_node.keys())[0]]
  merchant_node = merchantDF.loc[merchantDF['id']==merchant_id].to_dict(orient='index') #hoping to be a pd.Series
  merchant_node = merchant_node[list(merchant_node.keys())[0]]
  graph.add_edge(user_node,merchant_node,time=row['time'],relation_type=row['type'])

dill.dump(graph, open(output_dir,'wb'))

100%|██████████| 94876/94876 [04:28<00:00, 353.87it/s]


In [ ]:
graph.node_feature['user']

,id,type,mean_amt,min_amt,1Q_amt,2Q_amt,3Q_amt,max_amt,gender,city,state,zip,lat,long,city_pop,job,age
0,567868110212,user,83.283737,1.02,16.6500,53.205,107.6700,2713.44,F,Houston,TX,77007,29.7736,-95.4034,2906700,"Copywriter, advertising",38
1,371284100299909,user,57.144995,1.04,7.6350,31.960,72.8400,2283.19,F,Hedley,TX,79237,34.8698,-100.6806,513,Early years teacher,46
2,3518234918950662,user,89.386382,1.01,18.3050,54.280,115.6525,1793.41,F,Sherman,TX,75092,33.6372,-96.6184,46563,Systems developer,52
3,4425161475596168,user,71.959466,1.04,8.0900,51.645,85.5350,1415.74,F,San Angelo,TX,76905,31.4647,-100.3900,103927,Drilling engineer,60
4,4908846471916297,user,95.583577,1.01,19.1600,52.160,108.6950,12176.55,F,Grandview,TX,76050,32.2779,-97.2351,5875,"Radiographer, therapeutic",30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,6011935564104173,user,948.818182,311.58,922.3100,972.280,1075.6200,1124.34,F,Dallas,TX,75246,32.7948,-96.7697,1263321,Architect,73
62,6597888193422452,user,65.231555,1.01,8.5975,37.915,74.9375,5506.61,M,Holliday,TX,76366,33.6751,-98.6576,2317,Chief Technology Officer,58
63,3527063666782582,user,61.031577,1.00,11.0200,44.850,82.5400,654.18,F,Camden,TX,75934,30.9000,-94.7547,99,Colour technologist,57
64,4756039869079882102,user,59.669963,1.00,8.8725,47.640,83.3450,757.90,M,Gainesville,TX,76240,33.6547,-97.1583,26120,"Engineer, manufacturing",68
